# Summary
This script fetches the last 100 proposals and puts them in an Excel.

# Instructions
- to make this file yours, click **File -> Save a copy in Drive** (you only need to do this once), then work with the copied file
- set the `network` in the first code block (lower caps, only `polkadot` or `kusama`)
- in the menu above click **Runtime -> Run all**
  - an info box pops up: **Run anyway**
- wait half a minute (the wheels in the code boxes will stop turning when done)
- open the file browser on the left
  - download `{network}.xlsx` (if you don't see the file, click the refresh icon in the file browser)

# Notes
- Data is fetched from Polkassembly.
- USD prices of executed proposals are calculated to the exchange rate of the day of the last status change.
- Not every referendum gets a DOT value assigned from Polkassembly. E.g. Bounties are not counted, since the money is not spent. We also see proposals without value where we don't have an explanation yet, e.g. 465

In [181]:
network = "polkadot"
# network = "kusama"
explorer = "polkassembly"
# explorer = "subsquare"

if network == "polkadot":
  denomination_factor = 1e10
  ticker = "DOT-USD"
else:
  denomination_factor = 1e12
  ticker = "KSM-USD"

# Preconditions

In [182]:
import requests
import pandas as pd
import json
import datetime
import time

In [183]:
pip install pandas_datareader --upgrade

# Fetch Data

## Prices

In [184]:
# Import the yfinance. If you get module not found error the run !pip install yfinance from your Jupyter notebook
import yfinance as yf


def get_historic_dotusd_price():
  # 1. Get today's date
  today = datetime.datetime.now().strftime("%Y-%m-%d")

  # Get the data for the stock AAPL
  data = yf.download(ticker,'2020-08-20',today)
  return data

dotusd_historic_df = get_historic_dotusd_price()
dotusd_historic_df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-08-20,2.787857,3.077784,2.692896,2.900080,2.900080,48819867
2020-08-21,2.896923,3.090093,2.730919,2.875028,2.875028,49313137
2020-08-22,2.878570,4.510613,2.816762,4.484690,4.484690,485801096
2020-08-23,4.487058,4.487058,3.716271,3.967066,3.967066,320749483
2020-08-24,3.987413,4.783158,3.812670,4.602614,4.602614,407690171
...,...,...,...,...,...,...
2024-02-22,7.469288,7.661428,7.285482,7.482891,7.482891,239367257
2024-02-23,7.482491,7.636072,7.279380,7.563416,7.563416,249643272
2024-02-24,7.563381,7.885220,7.444860,7.809398,7.809398,237375316


In [185]:
def get_current_dotusd_price():
    url = 'https://api.coingecko.com/api/v3/simple/price?ids=polkadot&vs_currencies=usd'
    response = requests.get(url)
    data = response.json()
    return data['polkadot']['usd']

# Get the DOTUSD exchange rate
dotusd_price = get_current_dotusd_price()
print(f"DOT/USD exchange rate: {dotusd_price}")

DOT/USD exchange rate: 8.13


## Fetch Proposals from Polkassembly

In [186]:


# Define the URL
page_size = 100
track_status = "All"
url = f"https://api.polkassembly.io/api/v1/listing/on-chain-posts?page=1&proposalType=referendums_v2&listingLimit={page_size}&trackStatus={track_status}&sortBy=newest"
headers = {"x-network":network}

# Send a GET request to the URL
response = requests.get(url, headers=headers)


# Check if the request was successful
if response.status_code != 200:
    summary = "Failed to retrieve data. Status code: {}".format(response.status_code)
    print(summary)
    exit()




In [187]:

data = response.json()
# normalize into df
raw = pd.DataFrame(data)
raw_df = pd.json_normalize(raw['posts'])
raw

,count,posts
0,528,"{'beneficiaries': [], 'comments_count': 1, 'cr..."
1,528,"{'beneficiaries': [], 'comments_count': 2, 'cr..."
2,528,{'beneficiaries': [{'value': '14d84YbAXRwWcNJh...
3,528,"{'beneficiaries': [], 'comments_count': 0, 'cr..."
4,528,{'beneficiaries': [{'value': '15aVbh9j99DyEJo9...
...,...,...
95,528,{'beneficiaries': [{'value': '13YMK2eeopZtUNpe...
96,528,{'beneficiaries': [{'value': '15DogU3PvARucj3Q...
97,528,"{'beneficiaries': [], 'comments_count': 2, 'cr..."
98,528,"{'beneficiaries': [], 'comments_count': 1, 'cr..."


In [188]:
raw_df.columns

Index(['beneficiaries', 'comments_count', 'created_at', 'curator',
       'description', 'end', 'hash', 'identity', 'isSpam',
       'isSpamReportInvalid', 'method', 'parent_bounty_index', 'post_id',
       'proposalHashBlock', 'proposer', 'spam_users_count', 'status',
       'status_history', 'tags', 'timeline', 'title', 'track_no', 'type',
       'user_id', 'post_reactions.👍', 'post_reactions.👎', 'tally.ayes',
       'tally.nays', 'tally.support', 'topic.id', 'topic.name',
       'requestedAmount'],
      dtype='object')

In [189]:
raw_df["status"].unique()

array(['Deciding', 'Submitted', 'Executed', 'Rejected', 'ConfirmStarted',
       'DecisionDepositPlaced', 'Confirmed', 'TimedOut', 'ConfirmAborted',
       'Cancelled'], dtype=object)

# Transform


In [194]:
# | index | Title | Status | USD | DOT | Comment | Phase | End Time | Propose Time | Beneficiary | Category | Subcategory | Subsquare | Polkassembly | Subscan |


df = raw_df.copy()

from datetime import datetime

# Define your ID to Origin mapping
id_to_origin_mapping = {
    0: 'Root',
    1: 'Whitelisted Caller',
    10: 'Staking Admin',
    11: 'Treasurer',
    12: 'Lease Admin',
    13: 'Fellowship Admin',
    14: 'General Admin',
    15: 'Auction Admin',
    20: 'Referendum Canceller',
    21: 'Referendum Killer',
    30: 'Small Tipper',
    31: 'Big Tipper',
    32: 'Small Spender',
    33: 'Medium Spender',
    34: 'Big Spender'
}

def determine_usd_price(row):
  statuses_where_i_want_to_get_the_historic_price = ["Executed"]
  if row["status"] in statuses_where_i_want_to_get_the_historic_price:
    executed_date = row["last_status_change"] # pd.to_datetime(
    # Find the closest matching date in dotusd_historic_df
    closest_date = dotusd_historic_df.index.get_loc(executed_date, method='nearest')
    conversion_rate = dotusd_historic_df.iloc[closest_date]['Close']
    return row["DOT"] * conversion_rate
  else:
    return row["DOT"] * dotusd_price

# Function to format USD amounts
def format_currency(amount):
    if amount >= 1_000_000:
        return '{:.1f}m'.format(amount / 1_000_000)
    elif amount >= 1_000:
        return '{:.0f}k'.format(amount / 1_000)
    else:
        return '{:.0f}'.format(amount)

# Function to format date to quarter and year
def format_date(date):
    if pd.isnull(date):
        return None
    return f"{date.year}-{date.month:02d}-{date.day:02d}"

# Build columns
df["last_status_change"] = pd.to_datetime(df["status_history"].apply(lambda x: x[-1]["timestamp"] if len(x) > 0 else None))
df["last_update"] = df["last_status_change"].apply(format_date)
df["DOT"] = (pd.to_numeric(df["requestedAmount"]) / denomination_factor)
df["DOT_formatted"] = df["DOT"].apply(format_currency)
df["USD"] = df.apply(determine_usd_price, axis=1)
df["USD_formatted"] = df["USD"].apply(format_currency)
df["Status"] = df["status"]
df["index"] = df["post_id"].apply(lambda x:f'=HYPERLINK("https://{network}.{explorer}.io/referenda/{x}", {x})')


# Replace the 'Track' column with the mapping from IDs to Origin names
df["Track"] = df["track_no"].map(id_to_origin_mapping)



# More filtering
df = df.set_index("post_id")
df = df[["index", "title", "DOT", "USD", "Status", "Track", "last_update", "DOT_formatted", "USD_formatted"]]

#df[df["Status"] == "Executed"]
df


<ipython-input-194-52caf6371421>:32: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  closest_date = dotusd_historic_df.index.get_loc(executed_date, method='nearest')
<ipython-input-194-52caf6371421>:32: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  closest_date = dotusd_historic_df.index.get_loc(executed_date, method='nearest')


,index,title,DOT,USD,Status,Track,last_update,DOT_formatted,USD_formatted
post_id,,,,,,,,,
527,"=HYPERLINK(""https://polkadot.polkassembly.io/r...",Reimbursement for lost pool rewards,NaN,NaN,Deciding,Small Tipper,2024-02-27,nan,nan
526,"=HYPERLINK(""https://polkadot.polkassembly.io/r...",#40 Bounty proposal: Business Development Polk...,NaN,NaN,Deciding,Medium Spender,2024-02-26,nan,nan
525,"=HYPERLINK(""https://polkadot.polkassembly.io/r...",Polkadot marketing proposal in Chinese speakin...,10458.0,8.502354e+04,Submitted,Medium Spender,2024-02-26,10k,85k
524,"=HYPERLINK(""https://polkadot.polkassembly.io/r...",Transfer the Lease of Bifrost Parachain 3356 t...,NaN,NaN,Deciding,Lease Admin,2024-02-27,nan,nan
523,"=HYPERLINK(""https://polkadot.polkassembly.io/r...",Rising Karting Star Seeks Polkadot Partnership,8260.0,6.715380e+04,Deciding,Small Spender,2024-02-25,8k,67k
...,...,...,...,...,...,...,...,...,...
432,"=HYPERLINK(""https://polkadot.polkassembly.io/r...",Treasury proposal: DOT liquidity loan for Bifr...,500000.0,3.595746e+06,Executed,Big Spender,2024-02-09,500k,3.6m
431,"=HYPERLINK(""https://polkadot.polkassembly.io/r...",Polkadot Watchdogs Governance Monthly Incentiv...,100000.0,8.130000e+05,Rejected,Big Spender,2024-02-16,100k,813k
430,"=HYPERLINK(""https://polkadot.polkassembly.io/r...",Open HRMP channels between AssetHub and Hashed...,NaN,NaN,TimedOut,General Admin,2024-02-01,nan,nan


# Export
## Export to Excel

In [191]:
!pip install openpyxl


In [192]:
# Specify the filename
filename = f'{network}.xlsx'

# Export the DataFrame to an Excel file
df.to_excel(filename, engine='openpyxl', index=False)

print(f'DataFrame has been exported to {filename}')


DataFrame has been exported to polkadot.xlsx


# Ignore

In [193]:
output = '''
# Connect to Google Drive
from google.colab import auth
import gspread
from google.auth import default
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
# Read the Treasury Spreadsheet
import pandas as pd
worksheet = gc.open('Scrape').sheet1
rows = worksheet.get_all_values()
df = pd.DataFrame(rows[1:], columns=rows[0])
df
'''

raw_df[raw_df["requestedAmount"].isna()][["method", "post_id"]]

,method,post_id
0,batch_all,527
1,approve_bounty,526
3,batch_all,524
11,batch_all,516
12,approve_bounty,515
14,batch_all,513
16,batch_all,511
17,accept_curator,510
18,accept_curator,509
22,dispatch_whitelisted_call_with_preimage,505
